In [1]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder 
import os
from tensorflow import keras
import random
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import keras

In [5]:
import torch
import torchvision.transforms.functional as TF
import torch.nn as nn

In [2]:
temperature_data_path = 'data/temperature/'
height_data_path = 'data/height/'

temperature_files = os.listdir(temperature_data_path)
height_files = os.listdir(height_data_path)

In [3]:
temperature_data = []
height_data = []

for temperature_file in temperature_files:
    file_path = temperature_data_path + temperature_file
    data = pd.read_csv(file_path)
    temperature_data.append(data)
    
for height_file in height_files:
    file_path = height_data_path + height_file
    data = pd.read_csv(file_path)
    height_data.append(data)

In [42]:
temp_data_train = np.array(temperature_data[:1600])
temp_data_val = np.array(temperature_data[1600:1900])
temp_data_test = np.array(temperature_data[1900:])

height_data_train = np.array(height_data[:1600])
height_data_val = np.array(height_data[1600:1900])
height_data_test = np.array(height_data[1900:])

In [4]:
temperature_data_new = np.array(temperature_data)
height_data_new = np.array(height_data)

In [31]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

In [37]:
class UNET(nn.Module):
    def __init__(
            self, in_channels=1, out_channels=1, features=[64, 128, 256, 512],  # TRY IN CHANNELS = 1
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

In [40]:
def test():
    x = torch.randn((3, 1, 200, 201))
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    print(preds.shape)
    print(x.shape)
    assert preds.shape == x.shape

In [41]:
if __name__ == "__main__":
    test()

torch.Size([3, 1, 200, 201])
torch.Size([3, 1, 200, 201])
